# Exploración de Datos - Small Caps Trading System

Notebook para explorar los datos descargados de Polygon.io:
- Daily bars (1d)
- Hourly bars (1h)
- Tickers universe
- Corporate actions (splits, dividends)

In [3]:
pwd

'D:\\04_TRADING_SMALLCAPS\\notebooks'

In [24]:
import polars as pl
from pathlib import Path
import plotly.express as px
import plotly.graph_objects as go
from datetime import datetime, timedelta

In [25]:
# Paths
base = Path("..")
raw = base / "raw"
tickers_dir = raw / "reference"  # ← IMPORTANTE: reference, no tickers

In [26]:
# Cargar tickers
ticker_files = sorted(tickers_dir.glob("tickers_*.parquet"))
print(f"Archivos encontrados: {len(ticker_files)}")
if ticker_files:
    df_tickers = pl.read_parquet(ticker_files[-1])
    print(f"Tickers totales: {len(df_tickers):,}")

Archivos encontrados: 4
Tickers totales: 22,489


In [48]:
df_tickers.head()

ticker,name,market,locale,primary_exchange,type,active,currency_name,cik,composite_figi,share_class_figi,last_updated_utc,delisted_utc
str,str,str,str,str,str,bool,str,str,str,str,str,str
"""AAAP""","""Advanced Accelerator Applicati…","""stocks""","""us""","""XNAS""","""ADRC""",false,"""usd""","""0001611787""","""BBG00K6FMBQ8""","""BBG007K5CVB6""","""2024-12-03T20:51:58.376135Z""","""2018-02-12T05:00:00Z"""
"""AAB.WS""","""LEHMAN BROTHERS CURRENCY BASKE…","""stocks""","""us""","""XASE""",null,false,"""usd""",null,null,null,"""2025-01-16T17:32:57.087334Z""","""2008-02-11T05:00:00Z"""
"""AABA""","""Altaba Inc. Common Stock""","""stocks""","""us""","""XNAS""","""CS""",false,"""usd""","""0001011006""","""BBG000KB2D74""","""BBG001S8V781""","""2024-12-03T21:33:22.821052Z""","""2019-10-07T04:00:00Z"""
"""AABC""","""ACCESS ANYTIME BANCORP INC""","""stocks""","""us""",null,null,false,"""usd""","""0001024015""",null,null,"""2025-08-12T16:27:08.260299Z""","""2006-01-04T05:00:00Z"""
"""AAC""","""Ares Acquisition Corporation""","""stocks""","""us""","""XNYS""","""CS""",false,"""usd""","""0001829432""",null,null,"""2025-08-13T14:20:43.779961Z""","""2023-11-07T05:00:00Z"""


In [49]:
df_tickers.columns

['ticker',
 'name',
 'market',
 'locale',
 'primary_exchange',
 'type',
 'active',
 'currency_name',
 'cik',
 'composite_figi',
 'share_class_figi',
 'last_updated_utc',
 'delisted_utc']

In [44]:
# Distribución por tipo
if 'type' in df_tickers.columns:
    type_counts = df_tickers.group_by("type").agg(pl.len().alias("count")).sort("count", descending=True)
type_counts.head()

type,count
str,u32
null,6832
"""CS""",6225
"""SP""",2005
"""PFD""",1765
"""WARRANT""",1719


In [51]:
# Celda - Cargar daily bars (AAPL):
# Daily bars
bars_1d = raw / "market_data" / "bars" / "1d"
files_1d = sorted(bars_1d.glob("*.parquet")) if bars_1d.exists() else []
print(f"Archivos daily bars: {len(files_1d)}")

# Cargar AAPL
ticker = "AAPL"
file_1d = bars_1d / f"{ticker}.parquet"

if file_1d.exists():
    df_1d = pl.read_parquet(file_1d)
    print(f"\nTicker: {ticker}")
    print(f"Rows: {len(df_1d):,}")
    print(f"Date range: {df_1d['timestamp'].min()} -> {df_1d['timestamp'].max()}")
    df_1d.head(10)

Archivos daily bars: 3

Ticker: AAPL
Rows: 22
Date range: 2025-09-08 04:00:00+00:00 -> 2025-10-07 04:00:00+00:00


In [52]:
# Hourly bars
bars_1h = raw / "market_data" / "bars" / "1h"
file_1h = bars_1h / f"{ticker}.parquet"

if file_1h.exists():
    df_1h = pl.read_parquet(file_1h)
    print(f"Ticker: {ticker}")
    print(f"Rows: {len(df_1h):,}")
    print(f"Date range: {df_1h['timestamp'].min()} -> {df_1h['timestamp'].max()}")
    df_1h.head(10)

Ticker: AAPL
Rows: 356
Date range: 2025-09-08 08:00:00+00:00 -> 2025-10-08 11:00:00+00:00


In [53]:
# Comparación
if file_1d.exists() and file_1h.exists():
    print(f"Daily bars (1d): {len(df_1d):,} rows")
    print(f"Hourly bars (1h): {len(df_1h):,} rows")
    print(f"Ratio: {len(df_1h) / len(df_1d):.1f}x más granular")
    
    size_1d = file_1d.stat().st_size / 1024
    size_1h = file_1h.stat().st_size / 1024
    print(f"\nTamaño 1d: {size_1d:.1f} KB")
    print(f"Tamaño 1h: {size_1h:.1f} KB")

Daily bars (1d): 22 rows
Hourly bars (1h): 356 rows
Ratio: 16.2x más granular

Tamaño 1d: 3.9 KB
Tamaño 1h: 11.8 KB


## 1. Universo de Tickers

In [54]:
# Cargar último archivo de tickers
ticker_files = sorted(tickers_dir.glob("tickers_*.parquet"))
if ticker_files:
    df_tickers = pl.read_parquet(ticker_files[-1])
    print(f"Tickers totales: {len(df_tickers):,}")
    print(f"Archivo: {ticker_files[-1].name}")
    print(f"\nColumnas: {df_tickers.columns}")
    df_tickers.head(10)
else:
    print("No ticker files found")

Tickers totales: 22,489
Archivo: tickers_delisted_20251008.parquet

Columnas: ['ticker', 'name', 'market', 'locale', 'primary_exchange', 'type', 'active', 'currency_name', 'cik', 'composite_figi', 'share_class_figi', 'last_updated_utc', 'delisted_utc']


In [55]:
# Distribución por tipo de ticker
if 'type' in df_tickers.columns:
    type_counts = df_tickers.group_by("type").agg(pl.count().alias("count")).sort("count", descending=True)
    print(type_counts)
    
    # Gráfico
    fig = px.bar(type_counts.to_pandas(), x="type", y="count", 
                 title="Distribución de Tickers por Tipo")
    fig.show()

shape: (16, 2)
┌─────────┬───────┐
│ type    ┆ count │
│ ---     ┆ ---   │
│ str     ┆ u32   │
╞═════════╪═══════╡
│ null    ┆ 6832  │
│ CS      ┆ 6225  │
│ SP      ┆ 2005  │
│ PFD     ┆ 1765  │
│ WARRANT ┆ 1719  │
│ …       ┆ …     │
│ ETN     ┆ 203   │
│ ADRP    ┆ 15    │
│ ETS     ┆ 15    │
│ ADRR    ┆ 5     │
│ ETV     ┆ 5     │
└─────────┴───────┘


C:\Users\AlexJ\AppData\Local\Temp\ipykernel_45540\2487634203.py:3: DeprecationWarning: `pl.count()` is deprecated. Please use `pl.len()` instead.
(Deprecated in version 0.20.5)
  type_counts = df_tickers.group_by("type").agg(pl.count().alias("count")).sort("count", descending=True)


ModuleNotFoundError: No module named 'pandas'

In [56]:
# Distribución por market
if 'market' in df_tickers.columns:
    market_counts = df_tickers.group_by("market").agg(pl.count().alias("count")).sort("count", descending=True)
    print(market_counts)

shape: (1, 2)
┌────────┬───────┐
│ market ┆ count │
│ ---    ┆ ---   │
│ str    ┆ u32   │
╞════════╪═══════╡
│ stocks ┆ 22489 │
└────────┴───────┘


C:\Users\AlexJ\AppData\Local\Temp\ipykernel_45540\2022846949.py:3: DeprecationWarning: `pl.count()` is deprecated. Please use `pl.len()` instead.
(Deprecated in version 0.20.5)
  market_counts = df_tickers.group_by("market").agg(pl.count().alias("count")).sort("count", descending=True)


## 2. Daily Bars (1d)

In [57]:
# Listar archivos disponibles
files_1d = sorted(bars_1d.glob("*.parquet")) if bars_1d.exists() else []
print(f"Archivos daily bars: {len(files_1d)}")
if files_1d:
    print(f"Primeros 10: {[f.stem for f in files_1d[:10]]}")

Archivos daily bars: 3
Primeros 10: ['AAPL', 'GOOG', 'MSFT']


In [58]:
# Cargar un ticker ejemplo (AAPL)
ticker = "AAPL"
file_1d = bars_1d / f"{ticker}.parquet"

if file_1d.exists():
    df_1d = pl.read_parquet(file_1d)
    print(f"Ticker: {ticker}")
    print(f"Rows: {len(df_1d):,}")
    print(f"Date range: {df_1d['timestamp'].min()} -> {df_1d['timestamp'].max()}")
    print(f"\nColumnas: {df_1d.columns}")
    print(f"\nPrimeras filas:")
    df_1d.head(10)
else:
    print(f"File not found: {file_1d}")

Ticker: AAPL
Rows: 22
Date range: 2025-09-08 04:00:00+00:00 -> 2025-10-07 04:00:00+00:00

Columnas: ['volume', 'vwap', 'open', 'close', 'high', 'low', 'timestamp', 'transactions', 'symbol', 'date']

Primeras filas:


In [59]:
# Estadísticas básicas
if file_1d.exists():
    print("Estadísticas de precios:")
    df_1d.select(["close", "volume", "vwap"]).describe()

Estadísticas de precios:


In [60]:
# Gráfico de precio
if file_1d.exists():
    df_plot = df_1d.sort("timestamp").to_pandas()
    
    # Candlestick chart
    fig = go.Figure(data=[go.Candlestick(
        x=df_plot['timestamp'],
        open=df_plot['open'],
        high=df_plot['high'],
        low=df_plot['low'],
        close=df_plot['close']
    )])
    
    fig.update_layout(
        title=f"{ticker} - Daily Bars (1d)",
        yaxis_title="Price (USD)",
        xaxis_title="Date",
        height=600
    )
    fig.show()

ModuleNotFoundError: No module named 'pandas'

In [61]:
# Gráfico de volumen
if file_1d.exists():
    fig = px.bar(df_plot, x="timestamp", y="volume", 
                 title=f"{ticker} - Volume (Daily)")
    fig.update_layout(height=400)
    fig.show()

NameError: name 'df_plot' is not defined

## 3. Hourly Bars (1h)

In [62]:
# Listar archivos disponibles
files_1h = sorted(bars_1h.glob("*.parquet")) if bars_1h.exists() else []
print(f"Archivos hourly bars: {len(files_1h)}")
if files_1h:
    print(f"Primeros 10: {[f.stem for f in files_1h[:10]]}")

Archivos hourly bars: 3
Primeros 10: ['AAPL', 'GOOG', 'MSFT']


In [63]:
# Cargar mismo ticker (AAPL)
file_1h = bars_1h / f"{ticker}.parquet"

if file_1h.exists():
    df_1h = pl.read_parquet(file_1h)
    print(f"Ticker: {ticker}")
    print(f"Rows: {len(df_1h):,}")
    print(f"Date range: {df_1h['timestamp'].min()} -> {df_1h['timestamp'].max()}")
    print(f"\nColumnas: {df_1h.columns}")
    print(f"\nPrimeras filas:")
    df_1h.head(10)
else:
    print(f"File not found: {file_1h}")

Ticker: AAPL
Rows: 356
Date range: 2025-09-08 08:00:00+00:00 -> 2025-10-08 11:00:00+00:00

Columnas: ['volume', 'vwap', 'open', 'close', 'high', 'low', 'timestamp', 'transactions', 'symbol', 'date']

Primeras filas:


In [64]:
# Gráfico de precios (últimos 30 días)
if file_1h.exists():
    # Filtrar últimos 30 días
    cutoff = datetime.utcnow() - timedelta(days=30)
    df_recent = df_1h.filter(pl.col("timestamp") >= cutoff).sort("timestamp")
    df_plot_1h = df_recent.to_pandas()
    
    fig = go.Figure(data=[go.Candlestick(
        x=df_plot_1h['timestamp'],
        open=df_plot_1h['open'],
        high=df_plot_1h['high'],
        low=df_plot_1h['low'],
        close=df_plot_1h['close']
    )])
    
    fig.update_layout(
        title=f"{ticker} - Hourly Bars (1h) - Last 30 Days",
        yaxis_title="Price (USD)",
        xaxis_title="DateTime",
        height=600
    )
    fig.show()

C:\Users\AlexJ\AppData\Local\Temp\ipykernel_45540\2988734173.py:4: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  cutoff = datetime.utcnow() - timedelta(days=30)


SchemaError: could not evaluate '>=' comparison between series 'timestamp' of dtype: datetime[ms, UTC] and series 'literal' of dtype: datetime[μs]

## 4. Comparación: Daily vs Hourly

In [65]:
# Comparar cantidad de datos
if file_1d.exists() and file_1h.exists():
    print(f"Daily bars (1d): {len(df_1d):,} rows")
    print(f"Hourly bars (1h): {len(df_1h):,} rows")
    print(f"\nRatio: {len(df_1h) / len(df_1d):.1f}x más granular")
    
    # Tamaño de archivos
    size_1d = file_1d.stat().st_size / 1024  # KB
    size_1h = file_1h.stat().st_size / 1024  # KB
    print(f"\nTamaño 1d: {size_1d:.1f} KB")
    print(f"Tamaño 1h: {size_1h:.1f} KB")
    print(f"Ratio: {size_1h / size_1d:.1f}x")

Daily bars (1d): 22 rows
Hourly bars (1h): 356 rows

Ratio: 16.2x más granular

Tamaño 1d: 3.9 KB
Tamaño 1h: 11.8 KB
Ratio: 3.0x


## 5. Corporate Actions - Splits

In [66]:
# Cargar splits
split_files = sorted(splits_dir.glob("*.parquet")) if splits_dir.exists() else []
if split_files:
    df_splits = pl.read_parquet(split_files[-1])
    print(f"Total splits: {len(df_splits):,}")
    print(f"\nColumnas: {df_splits.columns}")
    print(f"\nÚltimos 10 splits:")
    df_splits.sort("execution_date", descending=True).head(10)
else:
    print("No splits files found")

No splits files found


## 6. Corporate Actions - Dividends

In [67]:
# Cargar dividends
dividend_files = sorted(dividends_dir.glob("*.parquet")) if dividends_dir.exists() else []
if dividend_files:
    df_divs = pl.read_parquet(dividend_files[-1])
    print(f"Total dividends: {len(df_divs):,}")
    print(f"\nColumnas: {df_divs.columns}")
    print(f"\nÚltimos 10 dividends:")
    df_divs.sort("ex_dividend_date", descending=True).head(10)
else:
    print("No dividend files found")

No dividend files found


In [68]:
# Dividends por ticker (top 20)
if dividend_files:
    top_div = df_divs.group_by("ticker").agg([
        pl.count().alias("num_dividends"),
        pl.col("cash_amount").sum().alias("total_amount")
    ]).sort("num_dividends", descending=True).head(20)
    
    print("Top 20 tickers por número de dividendos:")
    top_div

## 7. Exploración Libre

In [69]:
# Espacio para exploración adicional
